In [1]:
import os
os.chdir('/Users/akki/Desktop/USC/FreshmanYear/QBIO490/qbio_490_AkanshaSallakonda/analysis_data')

In [2]:
import cptac
cptac.list_datasets()

,Description,Data reuse status,Publication link
Dataset name,,,
Brca,breast cancer,no restrictions,https://pubmed.ncbi.nlm.nih.gov/33212010/
Ccrcc,clear cell renal cell carcinoma (kidney),no restrictions,https://pubmed.ncbi.nlm.nih.gov/31675502/
Colon,colorectal cancer,no restrictions,https://pubmed.ncbi.nlm.nih.gov/31031003/
Endometrial,endometrial carcinoma (uterine),no restrictions,https://pubmed.ncbi.nlm.nih.gov/32059776/
Gbm,glioblastoma,no restrictions,https://pubmed.ncbi.nlm.nih.gov/33577785/
Hnscc,head and neck squamous cell carcinoma,no restrictions,https://pubmed.ncbi.nlm.nih.gov/33417831/
Lscc,lung squamous cell carcinoma,no restrictions,https://pubmed.ncbi.nlm.nih.gov/34358469/
Luad,lung adenocarcinoma,no restrictions,https://pubmed.ncbi.nlm.nih.gov/32649874/
Ovarian,high grade serous ovarian cancer,no restrictions,https://pubmed.ncbi.nlm.nih.gov/27372738/


cptac warning: Your version of cptac (1.1.2) is out-of-date. Latest is 1.5.7. Please run 'pip install --upgrade cptac' to update it. (/Users/akki/miniconda3/envs/qbio_490_Akansha2/lib/python3.9/threading.py, line 917)


In [16]:
import pandas as pd
import numpy as np

cptac.download(dataset = "Ccrcc")

ccrcc = cptac.Ccrcc()
ccrcc.list_data()

Below are the dataframes contained in this dataset and their dimensions:

clinical
	194 rows
	171 columns
CNV
	110 rows
	19285 columns
followup
	352 rows
	27 columns
medical_history
	370 rows
	4 columns
methylation
	107 rows
	15885 columns
phosphoproteomics
	194 rows
	81550 columns
phosphoproteomics_gene
	194 rows
	6127 columns
proteomics
	194 rows
	11710 columns
somatic_mutation
	8350 rows
	3 columns
transcriptomics
	185 rows
	19275 columns


In [30]:
#masking to only have Stage I-III tumors

clinical_data = ccrcc.get_clinical()
stage_mask = np.where(clinical_data.tumor_stage_pathological != "Stage IV", True, False)
clinical_data_mid = clinical_data[stage_mask]
clinical_data = clinical_data_mid[clinical_data_mid.tumor_stage_pathological.notna()]

In [33]:
#Masking RNA data to reflect what data is available in clinical data
rna_data = ccrcc.get_transcriptomics()
rna_mask = rna_data.index.isin(clinical_data.index)
rna_data = rna_data[rna_mask]
rna_data = np.log2(rna_data) 
rna_data = rna_data.dropna(axis=1)
rna_data = rna_data.T.loc[~rna_data.T.index.duplicated(), :].T

/Users/akki/miniconda3/envs/qbio_490_Akansha2/lib/python3.9/site-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: divide by zero encountered in log2
  result = func(self.values, **kwargs)


In [32]:
#Masking protein data to reflect what data is available in clinical data
protein_data = ccrcc.get_proteomics()
protein_data.columns = protein_data.columns.get_level_values(0) 
protein_mask = protein_data.index.isin(clinical_data.index)
protein_data = protein_data[protein_mask]
protein_data = protein_data.dropna(axis=1)
protein_data = protein_data.T.loc[~protein_data.T.index.duplicated(), :].T

In [40]:
rna_mean = rna_data.mean()
protein_mean = protein_data.mean()

top_five_protein = np.absolute(protein_mean).sort_values(ascending=False).head(5)
print(top_five_protein)

top_five_RNA = np.absolute(rna_mean).sort_values(ascending=False).head(5)
print(top_five_RNA)

top_RNA_list = ["NUP98", "NMT1", "RAB8A", "TNPO3", "ZNF664"]
top_protein_list = ["MT1H", "MT1G", "MT1F", "MT1E", "MT1B"]

top_RNA_df = rna_data.loc[:, top_RNA_list]
top_protein_df = protein_data.loc[:, top_protein_list]

#Top 5 differentially expressed RNA genes : MMP8, SLC6A11, SLC5A11, IGFL2, SLC5A5
#Top 5 differentially expressed Proteins: MT1H, MT1G, MT1F, MT1E, MT1B

Name
MT1H    2.844733
MT1G    2.590020
MT1F    2.432565
MT1E    2.283102
MT1B    2.145197
dtype: float64
Name
MMP8       inf
SLC6A11    inf
SLC5A11    inf
IGFL2      inf
SLC5A5     inf
dtype: float64


In [41]:
#Combining genes and proteins of interest into one dataframe
top_ten_genes = pd.concat([top_RNA_df, top_protein_df], axis=1)

#Creating separate list of the patients' cancer stages

top_ten_genes["tumor_stage_pathological"] = clinical_data.loc[:, "tumor_stage_pathological"]

top_ten_genes

Name,NUP98,NMT1,RAB8A,TNPO3,ZNF664,MT1H,MT1G,MT1F,MT1E,MT1B,tumor_stage_pathological
Patient_ID,,,,,,,,,,,
C3L-00004,4.475587,4.709943,4.511866,4.538224,4.381721,-2.675872,-2.253525,-2.546547,-2.557182,-1.939419,Stage III
C3L-00010,4.099825,4.505286,4.568616,4.466137,4.387715,-4.050571,-4.060004,-2.921551,-3.591516,-3.363261,Stage I
C3L-00026,4.100547,4.361278,4.739195,4.825138,4.246069,-2.607363,-2.475965,-2.492909,-2.004049,-1.809835,Stage I
C3L-00079,4.391514,4.332005,4.433089,4.594001,4.313825,-1.735521,-2.525792,-3.036604,-2.514611,-1.763428,Stage III
C3L-00088,4.320449,4.248517,4.490979,4.416537,4.265659,-2.404961,-2.110608,-1.972565,-2.471242,-1.731799,Stage III
...,...,...,...,...,...,...,...,...,...,...,...
C3N-01646,4.128605,4.685423,4.673625,4.595805,4.921863,-2.183352,-2.047085,-2.162791,-2.431460,-1.742155,Stage III
C3N-01648,4.523754,4.319720,5.140829,4.569094,4.142063,-1.454984,-1.196218,-1.313544,-1.182120,-1.103926,Stage II
C3N-01649,4.597990,4.047711,4.184536,4.636023,4.694756,-2.214753,-2.534672,-2.485910,-2.172642,-2.360767,Stage III


In [42]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
unencoded_columns = patient_stages
encoded_columns = encoder.fit_transform(unencoded_columns)
patient_stages = encoded_columns

In [43]:
from sklearn.preprocessing import StandardScaler
from umap import UMAP


scaler = StandardScaler()

embedding_unscaled = UMAP().fit_transform(feature_df)

scaled_data = scaler.fit_transform(feature_df)
embedding_scaled = UMAP().fit_transform(scaled_data)

In [ ]:
Testing different algorithms for best fit

In [44]:
from sklearn.neighbors import KNeighborsClassifier # default number of neighbors looked at is 5
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

In [46]:
#single run

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

patient_stages = np.array(patient_stages)

classifier = KNeighborsClassifier() # your choice of which classifier to use

X_train, X_test, y_train, y_test = train_test_split(scaled_data, patient_stages, train_size=0.7)

classifier.fit(X_train, y_train.ravel())

y_pred = classifier.predict(X_test)

In [47]:
regressors_names = [
    'KNeighborsClassifier', 
    'DecisionTreeClassifier', 
    'MLPClassifier', 
    'GaussianNB'
]

regressors = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    MLPClassifier(),
    GaussianNB()
]

regressors_perf = {
    0: [],
    1: [],
    2: [],
    3: []
}

for n in range(10):
    X_train, X_test, y_train, y_test = train_test_split(scaled_data, patient_stages, train_size=0.7)
    for i in range(len(regressors)):
        model = regressors[i]
        model.fit(X_train, y_train.ravel())
        y_pred = model.predict(X_test)

        difference = y_pred - y_test
        regressors_perf[i].append(abs(np.mean(difference)))

print('\nAfter 10 simulations, the average error for each regressor is as follows:')
for i in regressors_perf:
    print(f'\t{regressors_names[i]} : {np.mean(regressors_perf[i])}')

/Users/akki/miniconda3/envs/qbio_490_Akansha2/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/akki/miniconda3/envs/qbio_490_Akansha2/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/akki/miniconda3/envs/qbio_490_Akansha2/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/akki/miniconda3/envs/qbio_490_Akansha2/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the 


After 10 simulations, the average error for each regressor is as follows:
	KNeighborsClassifier : 0.21666666666666665
	DecisionTreeClassifier : 0.2566666666666666
	MLPClassifier : 0.16
	GaussianNB : 0.13


/Users/akki/miniconda3/envs/qbio_490_Akansha2/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/akki/miniconda3/envs/qbio_490_Akansha2/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
#GaussianNB appears to be the best model as it has the least error. 